In [37]:
from PIL import Image, ImageDraw, ImageFont  
import numpy as np
import matplotlib.pyplot as plt
from copy import copy, deepcopy
from scipy import interpolate
import cv2 
from scipy.integrate import quad
from scipy.misc import derivative
import sympy as sp
import math
from math import sqrt, pi

In [2]:
# Parameters
font_type = 'C:/Windows/Fonts/Calibri/calibrib.ttf'
font_size = 49 # 49
character = 'A'
threshold_binarisation = 100

In [3]:
# Befehl muss zweimal ausgeführt werden (komisch)
%matplotlib qt
%matplotlib qt

In [4]:
img = Image.new('RGB', (50, 50), color = (0, 0, 0))   # (50,50)  
fnt = ImageFont.truetype(font_type, font_size)
d = ImageDraw.Draw(img)
d.text((0,0), character, font=fnt, fill=(255, 255, 255))
# img.show() 
# img.save('pil_text_font.bmp')
img = np.array(img)
plt.figure('img')
plt.imshow(img)
x,y,z = np.shape(img)
print(x,y,z)
img[img<threshold_binarisation] = 0
plt.figure('asdfasdf')
plt.imshow(img/255)

50 50 3


In [28]:
# img.astype(np.float32)
# img.astype(np.uint8)
# get the bounding rect
img_rect = deepcopy(img)
x, y, w, h = cv2.boundingRect(img_rect[:,:,1])
# print(x, y, w, h)
x_min = x-1
y_min = y-1
x_max = x+w
y_max = y+h
cv2.rectangle(img_rect, (x_min, y_min), (x_max, y_max), (0, 255, 0), 0)   # img wird überschrieben
plt.figure('img_rect')
plt.imshow(img_rect/255)

In [29]:
img_without_boundary = img_rect[y:y_max, x:x_max,:]
plt.figure('img_without_boundary')
plt.imshow(img_without_boundary/255)

In [30]:
# print(img_without_boundary)
y_img_without_boundary, x_img_without_boundary, z_img_without_boundary = np.shape(img_without_boundary)
# print(y_img_without_boundary, x_img_without_boundary, z_img_without_boundary)

In [31]:
tmp_x_min = int(-1*x_img_without_boundary/2)
tmp_x_max = int(np.ceil(x_img_without_boundary/2))
tmp_y_min = int(-1*y_img_without_boundary/2)
tmp_y_max = int(np.ceil(y_img_without_boundary/2))
# print(tmp_x_min)
# print(tmp_x_max)
# print(tmp_y_min)
# print(tmp_y_max)
x_mesh = np.arange(tmp_x_min, tmp_x_max, 1)
y_mesh = np.arange(tmp_y_min, tmp_y_max, 1)
# print(x_mesh)
# print(y_mesh)
xx, yy = np.meshgrid(x_mesh, np.flip(y_mesh,0), sparse=True)
yy = yy*1j
z = (yy+xx)
# print(z)
q = np.pi/180
c_meshgrid = z.imag*2*1j+3*z.real  # (z*np.cos(q)-np.sin(q)) / (z*np.sin(q)+np.cos(q*5))
# print(c_meshgrid)
y_meshgrid, x_meshgrid = c_meshgrid.shape
# print(y_meshgrid, x_meshgrid)
# c_meshgrid_reshape = c_meshgrid.reshape((y_meshgrid,x_meshgrid,1))

In [32]:
c_meshgrid_mask = deepcopy(c_meshgrid)
max_real_meshgrid_mask = int(np.max(c_meshgrid_mask.real))
max_imag_meshgrid_mask = int(np.max(c_meshgrid_mask.imag))
min_real_meshgrid_mask = int(np.min(c_meshgrid_mask.real))
min_imag_meshgrid_mask = int(np.min(c_meshgrid_mask.imag))
# print(max_real_meshgrid_mask)
# print(max_imag_meshgrid_mask)
# print(min_real_meshgrid_mask)
# print(min_imag_meshgrid_mask)
# print(c_meshgrid_mask.shape)

In [33]:
z_transform = np.array(np.zeros((max_imag_meshgrid_mask+1-min_imag_meshgrid_mask, max_real_meshgrid_mask-min_real_meshgrid_mask+1, 3)))
y_z_transform, x_z_transform, z_z_transform = z_transform.shape
# print(z_transform)
# print(y_z_transform, x_z_transform, z_z_transform)

In [34]:
for i in range(y_meshgrid):
    for j in range(x_meshgrid):
        for k in range(3):
            z_transform[int(max_imag_meshgrid_mask-c_meshgrid_mask[i,j].imag), int(c_meshgrid_mask[i,j].real-min_real_meshgrid_mask),:] = img_without_boundary[i,j,k]
        
#   Für Y-Achse => max-y
#   Für X-Achse => x-min

In [35]:
print(z_transform.shape)
print(img_without_boundary.shape)

(61, 82, 3)
(31, 28, 3)


In [36]:
plt.figure('transformation')
plt.imshow(z_transform/255)

# Without complex numbers <br>

In [52]:
x = sp.Symbol('x')
f = x**2   # shift half pixel to the right
f_derviate = sp.diff(f)
# print(type(f_derviate))
x = sp.Symbol(str(f_derviate))
# print(x)

In [53]:
# https://www.youtube.com/watch?v=QAqadzwo19Q
a = quad(lambda x: sqrt(x**2+1),0,5)
# print(type(x))
# print(a)
# print(type(float(a[0])))

In [54]:
index = np.array(np.zeros((1,9)))
# print(index.shape)
# print(index)

In [55]:
for i in range(0,5,1):
    tmp = quad(lambda x: sqrt(x**2+1),0,i)
    index[0,i] = round(float(tmp[0]))
#     print(index[0,i])
#     print(type(index[0,0]))

In [62]:
b = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
b[1:7:2]
np.array([1, 3, 5])
b[-2:10]
np.array([8, 9])
b[-3:3:-1]
np.array([7, 6, 5, 4])

array([7, 6, 5, 4])